In [ ]:
import pandas as pd
import scipy as sc
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from tqdm import tqdm_notebook
import nltk
import random
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('papers.csv')
aut=pd.read_csv('paper_authors.csv')
documents = list(df["paper_text"].values)[:3513]
documents = [" ".join([PorterStemmer().stem(word) for word in sentence.split(" ")]) for sentence in documents]
paper_id = list(df["id"].values)[:3513]
ar=[]
for i in range(len(documents)):
    ar.append([documents[i],paper_id[i]])
random.shuffle(ar)
documents=[]
paper_id=[]
for i in range(len(ar)):
    documents.append(ar[i][0])
    paper_id.append(ar[i][1])

In [ ]:
tfidf = TfidfVectorizer(stop_words=stopwords.words('english'))
X = tfidf.fit_transform(documents)

In [ ]:
papr_id = list(aut["paper_id"].values)
pap_id=np.asarray(papr_id)
aut_id = list(aut["author_id"].values)
split=3200
train_paperid=paper_id[:split]
test_paperid=paper_id[split:]

In [ ]:
auth=[]
for i in paper_id:
    ind=np.where(pap_id==i)[0]
    lis=[]
    for j in ind:
        lis.append(aut_id[j])
    auth.append(lis)

In [ ]:
authors=[]
for i in train_paperid:
    ind=np.where(pap_id==i)[0]
    for j in ind:
        m=0
        for k in authors:
            if (aut_id[j]==k):
                m=1
                break
        if (m==0):
            authors.append(aut_id[j])

In [ ]:
#9784 authors,4625 authors,3982 train authors
aut_pap=[]
for i in authors:
    lis=[]
    lis.append(i)
    for j in range(len(train_paperid)):
        for k in auth[j]:
            if (i==k):
                lis.append(j)
                break
    aut_pap.append(lis)

In [ ]:
tr=[]
for i in range(len(test_paperid)):
    m=0
    for j in auth[split+i]:
        for k in authors:
            if (j==k):
                m=1
                break
        if (m==1):
            break
    tr.append(m)

# Max Similarity

In [ ]:
lab=[]
for i in tqdm_notebook(range(len(test_paperid))):
    maxim=0
    maxind=-1
    if(tr[i]):
        for j in range(len(train_paperid)):
            sim=np.dot(X[j],X[split+i].T)
            if (sim>maxim):
                maxim=sim
                maxind=j
    lab.append(maxind)

In [ ]:
tot=0
for i in range(len(test_paperid)):
    y=0
    for k in auth[split+i]:
        if(lab[i]!=-1):
            for l in auth[lab[i]]:
                if(k==l):
                    tot=tot+1
                    y=1
                    break
            if (y==1):
                break
acc=tot/np.sum(tr)
print (acc)

# Concatenate (Accuracy + MRR)

In [ ]:
a,b=np.shape(X)
Xc=[]
for i in range(len(authors)):
    lis=np.zeros(b)
    for j in range(len(aut_pap[i])):
        if(j==0):
            continue
        lis=lis+(X[aut_pap[i][j]].todense())
    Xc.append(lis)
Xp= np.stack(Xc,axis=0)
Xr=sc.sparse.csr_matrix(Xp)

In [ ]:
def takeFirst(e):
    return e[0]
lab=[]
mrr=[]
for i in tqdm_notebook(range(len(test_paperid))):
    tot=0
    se=[]
    Q=0
    S=0
    if(tr[i]):
        for j in range(len(authors)):
            n2=np.linalg.norm(Xr[j].todense())
            sim=float((np.dot(Xr[j],X[split+i].T)/n2).todense())
            se.append((sim,j))
        se.sort(key=takeFirst,reverse=True)
        for k in range(len(authors)):
            for l in auth[split+i]:
                if(authors[se[k][1]]==l):
                    tot=tot+(1/(k+1))
                    Q=Q+1
                    S=S+1/Q
                    break
        if(S>0):
            mrr.append(tot/S)
        else:
            mrr.append(0)
        lab.append([se[0][1],se[1][1],se[2][1]])

In [ ]:
tot=0
q=0
for i in range(len(test_paperid)):
    if(tr[i]==0):
        q=q+1
        continue
    y=0
    for k in auth[split+i]:
        for j in range(3):
            if(k==authors[lab[i-q][j]]):
                tot=tot+1
                y=1
                break
        if (y==1):
            break
acc2=tot/np.sum(tr)
avermrr=np.sum(mrr)/len(mrr)
print (acc2,avermrr)

# Concatenate Top 3

In [ ]:
lab=[]
for i in tqdm_notebook(range(len(test_paperid))):
    maxim=np.zeros(3)
    maxind=[-1,-1,-1]
    if(tr[i]):
        for j in range(len(authors)):
            n2=np.linalg.norm(Xr[j].todense())
            sim=float((np.dot(Xr[j],X[split+i].T)/n2).todense())
            if (sim>maxim[0]):
                maxim[2]=maxim[1]
                maxim[1]=maxim[0]
                maxim[0]=sim
                maxind[2]=maxind[1]
                maxind[1]=maxind[0]
                maxind[0]=j
            elif (sim>maxim[1]):
                maxim[2]=maxim[1]
                maxim[1]=sim
                maxind[2]=maxind[1]
                maxind[1]=j
            elif (sim>maxim[2]):
                maxim[2]=sim
                maxind[2]=j
    lab.append(maxind)

In [ ]:
tot=0
for i in range(len(test_paperid)):
    y=0
    if(lab[i]!=-1):
        for k in auth[split+i]:
            for j in range(3):
                if(k==authors[lab[i][j]]):
                    tot=tot+1
                    y=1
                    break
            if (y==1):
                break
acc2=tot/np.sum(tr)
print (acc2)

# Average

In [ ]:
lab=[]
for i in tqdm_notebook(range(len(test_paperid))):
    maxim=0
    maxind=-1
    if(tr[i]):
        for j in range(len(authors)):
            sim=0
            for k in range(len(aut_pap[j])):
                if (k==0):
                    continue
                n2=np.linalg.norm(X[aut_pap[j][k]].todense())
                sim=sim+float((np.dot(X[aut_pap[j][k]],X[split+i].T)/n2).todense())
            sim=sim/(len(aut_pap[j]))
            if (sim>maxim):
                maxim=sim
                maxind=j
    lab.append(maxind)

In [ ]:
tot=0
for i in range(len(test_paperid)):
    y=0
    if(lab[i]!=-1):
        for k in auth[split+i]:
            if(k==authors[lab[i]]):
                tot=tot+1
                break
acc4=tot/np.sum(tr)
print (acc4)